<a href="https://colab.research.google.com/github/luricu/Python/blob/main/ConverterExcelToSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
# Load the Excel file
df = pd.read_excel("ANEXO 17.xlsx")

# Inspect the first few rows and column types
print(df.head())
print(df.info())

  ANEXO TRATAMENTO TRIBUTÁRIO    TIPO CÓDIGO FISCAL  \
0  XVII      IMPOSTO SELETIVO  NCM/SH         87.03   
1  XVII      IMPOSTO SELETIVO  NCM/SH       8704.21   
2  XVII      IMPOSTO SELETIVO  NCM/SH       8704.31   
3  XVII      IMPOSTO SELETIVO  NCM/SH    8704.41.00   
4  XVII      IMPOSTO SELETIVO  NCM/SH    8704.51.00   

             DESCRIÇÃO DO ITEM  
0                     Veículos  
1  Veículos (exceto caminhões)  
2  Veículos (exceto caminhões)  
3  Veículos (exceto caminhões)  
4  Veículos (exceto caminhões)  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ANEXO                  25 non-null     object
 1   TRATAMENTO TRIBUTÁRIO  25 non-null     object
 2   TIPO                   25 non-null     object
 3   CÓDIGO FISCAL          24 non-null     object
 4   DESCRIÇÃO DO ITEM      25 non-null     object
dtypes: obje

In [ ]:
# Assign the expected column names to the DataFrame
df.columns = ['ANEXO', 'TRATAMENTO_TRIBUTARIO', 'TIPO', 'CODIGO_FISCAL', 'DESCRICAO']
expanded_data = []

In [ ]:
for index, row in df.iterrows():
    # Split the CODIGO_FISCAL field by comma or semicolon
    codes = [code.strip() for code in str(row['CODIGO_FISCAL']).replace(';', ',').split(',')]

    # Iterate over each code and create a new row
    for code in codes:
        if code: # Check if the code is not empty
            expanded_data.append({
                'DFANEXO': row['ANEXO'],
                'DFTRATAMENTOTRIBUTARIO': row['TRATAMENTO_TRIBUTARIO'],
                'DFTIPO': row['TIPO'],
                'DFCODIGOFISCAL': code,
                'DFDESCRICAO': row['DESCRICAO']
            })

In [ ]:
# Create the final DataFrame for SQL generation
df_sql = pd.DataFrame(expanded_data)

In [ ]:
# Step 2: Generate the SQL INSERT statements
sql_statements = []
table_name = "TBANEXOREFORMATRIB"
columns = "(DFANEXO, DFTRATAMENTOTRIBUTARIO, DFTIPO, DFCODIGOFISCAL, DFDESCRICAO)"

In [ ]:
for index, row in df_sql.iterrows():
    # Ensure quotes are handled for string values and replace NaN (if any) with NULL
    values = []

    # Use the logical mapping for the VALUES order: DFANEXO, DFTRATAMENTOTRIBUTARIO, DFTIPO, DFCODIGOFISCAL, DFDESCRICAO
    # Note: The user's example in the prompt had the column order wrong in the VALUES list,
    # but the content itself implies the desired mapping. I will stick to the column order (DFANEXO, DFTRATAMENTOTRIBUTARIO, DFTIPO, DFCODIGOFISCAL, DFDESCRICAO) for the VALUES list.

    # 1. DFANEXO (I)
    values.append(f"'{row['DFANEXO']}'")

    # 2. DFTRATAMENTOTRIBUTARIO (REDUÇÃO A ZERO (0%))
    values.append(f"'{row['DFTRATAMENTOTRIBUTARIO']}'")

    # 3. DFTIPO (NCM/SH)
    values.append(f"'{row['DFTIPO']}'")

    # 4. DFCODIGOFISCAL (1006.20)
    values.append(f"'{row['DFCODIGOFISCAL']}'")

    # 5. DFDESCRICAO (Arroz...)
    # Escape single quotes in description
    description = str(row['DFDESCRICAO']).replace("'", "''")
    values.append(f"'{description}'")

    sql_statement = f"INSERT INTO {table_name} {columns} VALUES ({', '.join(values)});"
    sql_statements.append(sql_statement)

In [ ]:
# Join the SQL statements into a single output string
sql_output = "\n".join(sql_statements)

# Save the SQL output to a file
with open("ANEXO_XVII_INSERTS.sql", "w", encoding="utf-8") as f:
    f.write(sql_output)

In [ ]:
print(f"SQL statements successfully generated and saved to ANEXO_XVII_INSERTS.sql")
print("\nFirst 5 generated SQL statements:")
print('\n'.join(sql_statements[:5]))

SQL statements successfully generated and saved to ANEXO_XVII_INSERTS.sql

First 5 generated SQL statements:
INSERT INTO TBANEXOREFORMATRIB (DFANEXO, DFTRATAMENTOTRIBUTARIO, DFTIPO, DFCODIGOFISCAL, DFDESCRICAO) VALUES ('XVII', 'IMPOSTO SELETIVO', 'NCM/SH', '87.03', 'Veículos');
INSERT INTO TBANEXOREFORMATRIB (DFANEXO, DFTRATAMENTOTRIBUTARIO, DFTIPO, DFCODIGOFISCAL, DFDESCRICAO) VALUES ('XVII', 'IMPOSTO SELETIVO', 'NCM/SH', '8704.21', 'Veículos (exceto caminhões)');
INSERT INTO TBANEXOREFORMATRIB (DFANEXO, DFTRATAMENTOTRIBUTARIO, DFTIPO, DFCODIGOFISCAL, DFDESCRICAO) VALUES ('XVII', 'IMPOSTO SELETIVO', 'NCM/SH', '8704.31', 'Veículos (exceto caminhões)');
INSERT INTO TBANEXOREFORMATRIB (DFANEXO, DFTRATAMENTOTRIBUTARIO, DFTIPO, DFCODIGOFISCAL, DFDESCRICAO) VALUES ('XVII', 'IMPOSTO SELETIVO', 'NCM/SH', '8704.41.00', 'Veículos (exceto caminhões)');
INSERT INTO TBANEXOREFORMATRIB (DFANEXO, DFTRATAMENTOTRIBUTARIO, DFTIPO, DFCODIGOFISCAL, DFDESCRICAO) VALUES ('XVII', 'IMPOSTO SELETIVO', 'NCM/SH